<img style="float: center;" src='https://github.com/STScI-MIRI/MRS-ExampleNB/raw/main/assets/banner1.png' alt="stsci_logo" width="900px"/> 

<a id="title_ID"></a>
# MIRI MRS Batch Processing Notebook #

**Author**: David Law, AURA Associate Astronomer, MIRI branch
<br>
**Last Updated**: July 13, 2022
<br>
**Pipeline Version**: 1.6.0+ (unreleased)

The purpose of this notebook is to provide a framework for batch processing of MIRI MRS data through all three pipeline stages.  Data is assumed to be located in two Observation folders (science and background) according to paths set up below.

This example is set up to use observations from APT program 2732 (Stephan's Quintet ERO observations) using a standard 4-pt dither with dedicated background in all three grating settings.  Input data for this notebook can be obtained by downloading the 'uncal' files from MAST and placing them in directories set up below.  

<hr style="border:1px solid gray"> </hr>

1.<font color='white'>-</font>Configuration <a class="anchor" id="intro"></a>
------------------

In [1]:
# Set parameters to be changed here.
# It should not be necessary to edit cells below this in general unless modifying pipeline processing steps.

import sys,os, pdb

# CRDS context (if overriding)
#%env CRDS_CONTEXT jwst_0771.pmap

# Point to where the uncalibrated FITS files are from the science observation
input_dir = '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/uncal/'

# Point to where you want the output science results to go
output_dir = '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/'

# Point to where the uncalibrated FITS files are from the background observation
# If no background observation, leave this blank
input_bgdir = '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/uncal/'

# Point to where the output background observations should go
# If no background observation, leave this blank
output_bgdir = '/Users/dlaw/FlightData/APT2732_quintet/Data/Obs05/'

# Whether or not to run a given pipeline stage
# Science and background are processed independently through det1+spec2, and jointly in spec3

# Science processing
dodet1=True
dospec2=True
dospec3=True

# Background processing
if (input_bgdir != ''):
    dodet1bg=True
    dospec2bg=True
else:
    dodet1bg=False
    dospec2bg=False

<hr style="border:1px solid gray"> </hr>

2.<font color='white'>-</font>Imports and setup <a class="anchor" id="intro"></a>
------------------

In [2]:
# Now let's use the entire available screen width for the notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

/var/folders/lg/jw_brb093bl9n39fbgn1cw080001fw/T/ipykernel_35786/1813087124.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
# Basic system utilities for interacting with files
import glob
import time
import shutil
import warnings
import zipfile
import urllib.request

# Astropy utilities for opening FITS and ASCII files
from astropy.io import fits
from astropy.io import ascii
from astropy.utils.data import download_file
# Astropy utilities for making plots
from astropy.visualization import (LinearStretch, LogStretch, ImageNormalize, ZScaleInterval)

# Numpy for doing calculations
import numpy as np

# Matplotlib for making plots
import matplotlib.pyplot as plt
from matplotlib import rc

In [27]:
# Import the base JWST package
import jwst

In [5]:
# JWST pipelines (encompassing many steps)
from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Spec2Pipeline
from jwst.pipeline import Spec3Pipeline

# JWST pipeline utilities
from jwst import datamodels # JWST datamodels
from jwst.associations import asn_from_list as afl # Tools for creating association files
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase # Definition of a Lvl2 association file
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base # Definition of a Lvl3 association file

from stcal import dqflags # Utilities for working with the data quality (DQ) arrays

In [6]:
# Output subdirectories to keep science data products organized
det1_dir = os.path.join(output_dir, 'stage1/') # Detector1 pipeline outputs will go here
spec2_dir = os.path.join(output_dir, 'stage2/') # Spec2 pipeline outputs will go here
spec3_dir = os.path.join(output_dir, 'stage3/') # Spec3 pipeline outputs will go here

# We need to check that the desired output directories exist, and if not create them
if not os.path.exists(det1_dir):
    os.makedirs(det1_dir)
if not os.path.exists(spec2_dir):
    os.makedirs(spec2_dir)
if not os.path.exists(spec3_dir):
    os.makedirs(spec3_dir)

In [7]:
# Output subdirectories to keep background data products organized
det1_bgdir = os.path.join(output_bgdir, 'stage1/') # Detector1 pipeline outputs will go here
spec2_bgdir = os.path.join(output_bgdir, 'stage2/') # Spec2 pipeline outputs will go here

# We need to check that the desired output directories exist, and if not create them
if (output_bgdir != ''):
    if not os.path.exists(det1_bgdir):
        os.makedirs(det1_bgdir)
    if not os.path.exists(spec2_bgdir):
        os.makedirs(spec2_bgdir)

In [8]:
# Start a timer to keep track of runtime
time0 = time.perf_counter()

<hr style="border:1px solid gray"> </hr>

3.<font color='white'>-</font>Detector1 Pipeline <a class="anchor" id="det1"></a>
------------------

<div class="alert alert-block alert-warning">
In this section we process our simulated data through the Detector1 pipeline to create Lvl2a data products (i.e., uncalibrated slope images).
See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html
</div>

In [9]:
# First we'll define a function that will call the detector1 pipeline with our desired set of parameters
# We won't enumerate the individual steps
def rundet1(filename, outdir):
    print(filename)
    det1 = Detector1Pipeline() # Instantiate the pipeline
    det1.output_dir = outdir # Specify where the output should go
    
    # Overrides for whether or not certain steps should be skipped
    #det1.dq_init.skip = False
    #det1.saturation.skip = False
    #det1.firstframe.skip = False
    #det1.lastframe.skip = False
    #det1.reset.skip = False
    #det1.linearity.skip = False
    #det1.rscd.skip = False
    #det1.dark_current.skip = False
    #det1.refpix.skip = False
    #det1.jump.skip = False
    #det1.ramp_fit.skip = False
    #det1.gain_scale.skip = False
    
    # Bad pixel mask overrides
    #det1.dq_init.override_mask = 'myfile.fits'

    # Saturation overrides
    #et1.saturation.override_saturation = 'myfile.fits'
    
    # Reset overrides
    #det1.reset.override_reset = 'myfile.fits'
        
    # Linearity overrides
    #det1.linearity.override_linearity = 'myfile.fits'

    # RSCD overrides
    #det1.rscd.override_rscd = 'myfile.fits'
        
    # DARK overrides
    #det1.dark_current.override_dark = 'myfile.fits'
        
    # GAIN overrides
    #det1.jump.override_gain = 'myfile.fits'
    #det1.ramp_fit.override_gain = 'myfile.fits'
                
    # READNOISE overrides
    #det1.jump.override_readnoise = 'myfile.fits'
    #det1.ramp_fit.override_readnoise = 'myfile.fits'
        
    det1.save_results = True # Save the final resulting _rate.fits files
    det1(filename) # Run the pipeline on an input list of files

In [10]:
# Now let's look for input files of the form *uncal.fits from the science observation
sstring = input_dir + 'jw*mirifu*uncal.fits'
lvl1b_files = sorted(glob.glob(sstring))
print('Found ' + str(len(lvl1b_files)) + ' science input files to process')

Found 24 science input files to process


In [11]:
# Run the pipeline on these input files by a simple loop over our pipeline function
if dodet1:
    for file in lvl1b_files:
        rundet1(file, det1_dir)
else:
    print('Skipping Detector1 processing')

Skipping Detector1 processing


In [12]:
# Now let's look for input files of the form *uncal.fits from the background observation
sstring = input_bgdir + 'jw*mirifu*uncal.fits'
lvl1b_files = sorted(glob.glob(sstring))
print('Found ' + str(len(lvl1b_files)) + ' background input files to process')

Found 12 background input files to process


In [13]:
# Run the pipeline on these input files by a simple loop over our pipeline function
if dodet1bg:
    for file in lvl1b_files:
        rundet1(file, det1_bgdir)
else:
    print('Skipping Detector1 processing')

Skipping Detector1 processing


In [14]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")

Runtime so far: 0.0383 seconds


4.<font color='white'>-</font>Spec2 Pipeline <a class="anchor" id="spec2"></a>
------------------

<div class="alert alert-block alert-warning">
In this section we process our simulated data through the Spec2 pipeline in order to produce Lvl2b data products (i.e., calibrated slope images and quick-look data cubes and 1d spectra).  
See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec2.html
</div>

In [15]:
# Define a function that will call the spec2 pipeline with our desired set of parameters
# We'll list the individual steps just to make it clear what's running
def runspec2(filename, outdir):
    spec2 = Spec2Pipeline()
    spec2.output_dir = outdir

    # Assign_wcs overrides
    #spec2.assign_wcs.override_distortion = 'myfile.asdf'
    #spec2.assign_wcs.override_regions = 'myfile.asdf'
    #spec2.assign_wcs.override_specwcs = 'myfile.asdf'
    #spec2.assign_wcs.override_wavelengthrange = 'myfile.asdf'

    # Flatfield overrides
    #spec2.flat.override_flat = 'myfile.fits'
        
    # Straylight overrides
    #spec2.straylight.override_mrsxartcorr = 'myfile.fits'
        
    # Fringe overrides
    #spec2.fringe.override_fringe = 'myfile.fits'
    
    # Photom overrides
    #spec2.photom.override_photom = 'myfile.fits'

    # Cubepar overrides
    #spec2.cube_build.override_cubepar = 'myfile.fits'
        
    # Extract1D overrides
    #spec2.extract1d.override_extract1d = 'myfile.asdf'
    #spec2.extract1d.override_apcorr = 'myfile.asdf'
        
    # Overrides for whether or not certain steps should be skipped
    #spec2.assign_wcs.skip = False
    #spec2.bkg_subtract.skip = True
    #spec2.flat_field.skip = False
    #spec2.srctype.skip = False
    #spec2.straylight.skip = False
    #spec2.fringe.skip = False
    #spec2.photom.skip = False
    #spec2.cube_build.skip = False
    #spec2.extract_1d.skip = False
    
    # Some cube building options
    #spec2.cube_build.weighting='drizzle'
    #spec2.cube_build.coord_system='ifualign' # If aligning cubes with IFU axes instead of sky
      
    spec2.save_results = True
    spec2(filename)

In [16]:
# Look for uncalibrated science slope files from the Detector1 pipeline
sstring = det1_dir + 'jw*mirifu*rate.fits'
ratefiles = sorted(glob.glob(sstring))
ratefiles=np.array(ratefiles)
print('Found ' + str(len(ratefiles)) + ' input files to process')

Found 24 input files to process


In [17]:
if dospec2:
    for file in ratefiles:
        runspec2(file, spec2_dir)
else:
    print('Skipping Spec2 processing')

Skipping Spec2 processing


In [18]:
# Look for uncalibrated background slope files from the Detector1 pipeline
sstring = det1_bgdir + 'jw*mirifu*rate.fits'
ratefiles = sorted(glob.glob(sstring))
ratefiles=np.array(ratefiles)
print('Found ' + str(len(ratefiles)) + ' input files to process')

Found 12 input files to process


In [19]:
if dospec2bg:
    for file in ratefiles:
        runspec2(file, spec2_bgdir)
else:
    print('Skipping Spec2 processing')

Skipping Spec2 processing


In [20]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")

Runtime so far: 0.0816 seconds


5.<font color='white'>-</font>Spec3 Pipeline: Default configuration (4 per-channel cubes)<a class="anchor" id="spec3"></a>
------------------

<div class="alert alert-block alert-warning">
Here we'll run the Spec3 pipeline to produce a composite data cube from all dithered exposures.
We will need to create an association file from all science and background data in order for the pipeline to use them appropriately.

A word of caution: the data cubes created by the JWST pipeline are in SURFACE BRIGHTNESS units (MJy/steradian), not flux units.  What that means is that if you intend to sum spectra within an aperture you need to be sure to multiply by the pixel area in steradians first in order to get a spectrum in flux units (the PIXAR_SR keyword can be found in the SCI extension header).  This correction is already build into the pipeline Extract1D algorithm.

See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec3.html
    
</div>

In [21]:
# Define a useful function to write out a Lvl3 association file from an input list
# Note that any background exposures have to be of type x1d.
def writel3asn(scifiles, bgfiles, asnfile, prodname):
    # Define the basic association of science files
    asn = afl.asn_from_list(scifiles, rule=DMS_Level3_Base, product_name=prodname)
        
    # Add background files to the association
    nbg=len(bgfiles)
    for ii in range(0,nbg):
        asn['products'][0]['members'].append({'expname': bgfiles[ii], 'exptype': 'background'})
        
    # Write the association to a json file
    _, serialized = asn.dump()
    with open(asnfile, 'w') as outfile:
        outfile.write(serialized)

In [22]:
# Find and sort all of the input files

# Science Files need the cal.fits files
sstring = spec2_dir + 'jw*mirifu*cal.fits'
calfiles = np.array(sorted(glob.glob(sstring)))

# Background Files need the x1d.fits files
sstring = spec2_bgdir + 'jw*mirifu*x1d.fits'
bgfiles = np.array(sorted(glob.glob(sstring)))

print('Found ' + str(len(calfiles)) + ' science files to process')
print('Found ' + str(len(bgfiles)) + ' background files to process')

Found 24 science files to process
Found 12 background files to process


In [23]:
# Make an association file that includes all of the different exposures
asnfile=os.path.join(output_dir, 'l3asn.json')
if dospec3:
    writel3asn(calfiles, bgfiles, asnfile, 'Level3')

In [24]:
# Define a function that will call the spec3 pipeline with our desired set of parameters
# This is designed to run on an association file
def runspec3(filename):
    # This initial setup is just to make sure that we get the latest parameter reference files
    # pulled in for our files.  This is a temporary workaround to get around an issue with
    # how this pipeline calling method works.
    crds_config = Spec3Pipeline.get_config_from_reference(filename)
    spec3 = Spec3Pipeline.from_config_section(crds_config)
    
    spec3.output_dir = spec3_dir
    #spec3.save_results = True
    
    # Cube building configuration options
    # spec3.cube_build.output_file = 'bandcube' # Custom output name
    # spec3.cube_build.output_type = 'band' # 'band', 'channel', or 'multi' type cube output
    # spec3.cube_build.channel = '1' # Build everything from just channel 1 into a single cube (we could also choose '2','3','4', or 'ALL')
    # spec3.cube_build.weighting = 'drizzle' # 'emsm' or 'drizzle'
    # spec3.cube_build.coord_system = 'ifualign' # 'ifualign', 'skyalign', or 'internal_cal'
    # spec3.cube_build.scale1 = 0.5 # Output cube spaxel scale (arcsec) in dimension 1 if setting it by hand
    # spec3.cube_build.scale2 = 0.5 # Output cube spaxel scale (arcsec) in dimension 2 if setting it by hand
    # spec3.cube_build.scalew = 0.002 # Output cube spaxel size (microns) in dimension 3 if setting it by hand
    
    # Overrides for whether or not certain steps should be skipped
    #spec3.assign_mtwcs.skip = False
    #spec3.master_background.skip = True
    #spec3.outlier_detection.skip = True
    #spec3.mrs_imatch.skip = True
    #spec3.cube_build.skip = False
    #spec3.extract_1d.skip = False
    
    # Cubepar overrides
    #spec3.cube_build.override_cubepar = 'myfile.fits'
        
    # Extract1D overrides
    #spec3.extract1d.override_extract1d = 'myfile.asdf'
    #spec3.extract1d.override_apcorr = 'myfile.asdf'

    spec3(filename)

In [25]:
if dospec3:
    runspec3(asnfile)
else:
    print('Skipping Spec3 processing')

2022-07-14 08:39:37,494 - stpipe - INFO - PARS-OUTLIERDETECTIONSTEP parameters found: /grp/crds/jwst/references/jwst/jwst_miri_pars-outlierdetectionstep_0051.asdf
2022-07-14 08:39:37,596 - stpipe - INFO - PARS-SPEC3PIPELINE parameters found: /grp/crds/jwst/references/jwst/jwst_miri_pars-spec3pipeline_0002.asdf
2022-07-14 08:39:37,628 - stpipe.Spec3Pipeline - INFO - Spec3Pipeline instance created.
2022-07-14 08:39:37,630 - stpipe - CRITICAL - config: {'skip': False, 'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True, 'save_results': False, 'suffix': 'assign_mtwcs', 'search_output_file': True, 'input_dir': None}


2022-07-14 08:39:37,631 - stpipe.Spec3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2022-07-14 08:39:37,633 - stpipe - CRITICAL - config: {'skip': False, 'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_u

2022-07-14 08:39:39,421 - stpipe.Spec3Pipeline - INFO - Prefetching reference files for dataset: 'jw02732004001_02101_00001_mirifulong_cal.fits' reftypes = ['apcorr', 'area', 'cubepar', 'drizpars', 'extract1d', 'photom', 'speckernel', 'specprofile', 'spectrace', 'wavemap']
2022-07-14 08:39:39,491 - stpipe.Spec3Pipeline - INFO - Prefetch for APCORR reference file is '/grp/crds/jwst/references/jwst/jwst_miri_apcorr_0002.asdf'.
2022-07-14 08:39:39,493 - stpipe.Spec3Pipeline - INFO - Prefetch for AREA reference file is 'N/A'.
2022-07-14 08:39:39,493 - stpipe.Spec3Pipeline - INFO - Prefetch for CUBEPAR reference file is '/grp/crds/jwst/references/jwst/jwst_miri_cubepar_0007.fits'.
2022-07-14 08:39:39,495 - stpipe.Spec3Pipeline - INFO - Prefetch for DRIZPARS reference file is '/grp/crds/jwst/references/jwst/jwst_miri_drizpars_0001.fits'.
2022-07-14 08:39:39,496 - stpipe.Spec3Pipeline - INFO - Prefetch for EXTRACT1D reference file is '/grp/crds/jwst/references/jwst/jwst_miri_extract1d_0003.as

2022-07-14 08:43:53,203 - stpipe.Spec3Pipeline.outlier_detection - INFO - Cube covers channel, subchannel: 1, medium
2022-07-14 08:43:53,573 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixels not mapped to output plane: 3029 with wavelength below 5.6579998474917375
2022-07-14 08:43:53,574 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixels not mapped to output plane: 2796 with wavelength above 6.6329998934525065
2022-07-14 08:43:54,843 - stpipe.Spec3Pipeline.outlier_detection - INFO - Number of spectral tear planes adjusted: 0
2022-07-14 08:43:58,476 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixels not mapped to output plane: 3029 with wavelength below 5.6579998474917375
2022-07-14 08:43:58,477 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixels not mapped to output plane: 2796 with wavelength above 6.6329998934525065
2022-07-14 08:43:59,717 - stpipe.Spec3Pipeline.outlier_detection - INFO - Number of spectr

2022-07-14 08:44:58,342 - stpipe.Spec3Pipeline.outlier_detection - INFO - Update S_REGION to POLYGON ICRS  339.013646693 33.974437759 339.013646693 33.976993315 339.016594427 33.976993315 339.016594427 33.974437759
2022-07-14 08:44:58,345 - stpipe.Spec3Pipeline.outlier_detection - INFO - Update S_REGION to POLYGON ICRS  339.013646693 33.974437759 339.013646693 33.976993315 339.016594427 33.976993315 339.016594427 33.974437759
2022-07-14 08:44:58,349 - stpipe.Spec3Pipeline.outlier_detection - INFO - Update S_REGION to POLYGON ICRS  339.013646693 33.974437759 339.013646693 33.976993315 339.016594427 33.976993315 339.016594427 33.974437759
2022-07-14 08:44:59,047 - stpipe.Spec3Pipeline.outlier_detection - INFO - Information on Blotting
2022-07-14 08:44:59,048 - stpipe.Spec3Pipeline.outlier_detection - INFO - Working with instrument MIRI
2022-07-14 08:44:59,049 - stpipe.Spec3Pipeline.outlier_detection - INFO - Shape of sky cube 45.000000 47.000000 641.000000
2022-07-14 08:44:59,049 - stpip

2022-07-14 08:45:51,464 - stpipe.Spec3Pipeline.outlier_detection - INFO - Blotting back to jw02732004001_02105_00002_mirifushort_cal.fits
2022-07-14 08:45:57,201 - stpipe.Spec3Pipeline.outlier_detection - INFO - Blotting back to jw02732004001_02105_00003_mirifushort_cal.fits
2022-07-14 08:46:02,977 - stpipe.Spec3Pipeline.outlier_detection - INFO - Blotting back to jw02732004001_02105_00004_mirifushort_cal.fits
2022-07-14 08:46:06,660 - stpipe.CubeBuildStep - INFO - CubeBuildStep instance created.
2022-07-14 08:46:06,661 - stpipe.CubeBuildStep - INFO - Starting IFU Cube Building Step
2022-07-14 08:46:06,661 - stpipe.CubeBuildStep - INFO - Input interpolation: pointcloud
2022-07-14 08:46:06,662 - stpipe.CubeBuildStep - INFO - Coordinate system to use: skyalign
2022-07-14 08:46:06,662 - stpipe.CubeBuildStep - INFO - Weighting method for point cloud: emsm
2022-07-14 08:46:06,663 - stpipe.CubeBuildStep - INFO - Power weighting distance: 2.0
2022-07-14 08:46:06,664 - stpipe.CubeBuildStep - I

...

2022-07-14 08:51:21,900 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixels not mapped to output plane: 235 with wavelength below 13.284499961649999
2022-07-14 08:51:21,901 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixels not mapped to output plane: 1084 with wavelength above 15.527499981457368
2022-07-14 08:51:23,242 - stpipe.Spec3Pipeline.outlier_detection - INFO - Number of spectral tear planes adjusted: 0
2022-07-14 08:51:26,372 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixels not mapped to output plane: 235 with wavelength below 13.284499961649999
2022-07-14 08:51:26,373 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixels not mapped to output plane: 1084 with wavelength above 15.527499981457368
2022-07-14 08:51:27,728 - stpipe.Spec3Pipeline.outlier_detection - INFO - Number of spectral tear planes adjusted: 0
2022-07-14 08:51:30,890 - stpipe.Spec3Pipeline.outlier_detection - INFO - # of detector pixe

2022-07-14 08:52:19,312 - stpipe.Spec3Pipeline.outlier_detection - INFO - Saved model in /Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage3/jw02732004001_02103_00001_mirifushort_a3001_crf.fits
2022-07-14 08:52:20,865 - stpipe.Spec3Pipeline.outlier_detection - INFO - Saved model in /Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage3/jw02732004001_02103_00002_mirifulong_a3001_crf.fits
2022-07-14 08:52:22,837 - stpipe.Spec3Pipeline.outlier_detection - INFO - Saved model in /Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage3/jw02732004001_02103_00002_mirifushort_a3001_crf.fits
2022-07-14 08:52:24,399 - stpipe.Spec3Pipeline.outlier_detection - INFO - Saved model in /Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage3/jw02732004001_02103_00003_mirifulong_a3001_crf.fits
2022-07-14 08:52:26,327 - stpipe.Spec3Pipeline.outlier_detection - INFO - Saved model in /Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage3/jw02732004001_02103_00003_mirifushort_a3001_crf.fits
2022-07

2022-07-14 08:54:11,818 - stpipe.Spec3Pipeline.cube_build - INFO - # of detector pixels not mapped to output plane: 4164 with wavelength above 7.651829253596442
2022-07-14 08:54:21,878 - stpipe.Spec3Pipeline.cube_build - INFO - # of detector pixels not mapped to output plane: 4164 with wavelength above 7.651829253596442
2022-07-14 08:54:31,937 - stpipe.Spec3Pipeline.cube_build - INFO - # of detector pixels not mapped to output plane: 4164 with wavelength above 7.6518292535956
2022-07-14 08:54:41,946 - stpipe.Spec3Pipeline.cube_build - INFO - # of detector pixels not mapped to output plane: 4164 with wavelength above 7.6518292535956
2022-07-14 08:54:50,823 - stpipe.Spec3Pipeline.cube_build - INFO - Average # of holes/wavelength plane is < 1
2022-07-14 08:54:50,824 - stpipe.Spec3Pipeline.cube_build - INFO - Total # of holes for IFU cube is : 0
2022-07-14 08:54:50,859 - stpipe.Spec3Pipeline.cube_build - INFO - Number of spectral tear planes adjusted: 0
2022-07-14 08:54:54,250 - stpipe.Spe

2022-07-14 08:57:50,925 - stpipe.Spec3Pipeline.cube_build - INFO - Cube Geometry:
2022-07-14 08:57:50,926 - stpipe.Spec3Pipeline.cube_build - INFO - axis#  Naxis  CRPIX    CRVAL      CDELT(arcsec)  Min & Max (xi, eta arcsec)
2022-07-14 08:57:50,926 - stpipe.Spec3Pipeline.cube_build - INFO - Axis 1    47  24.00 -20.98485572   0.20000000  -4.70000007   4.70000007
2022-07-14 08:57:50,927 - stpipe.Spec3Pipeline.cube_build - INFO - Axis 2    47  24.00  33.97571555   0.20000000  -4.70000007   4.70000007
2022-07-14 08:57:50,928 - stpipe.Spec3Pipeline.cube_build - INFO - axis#  Naxis  CRPIX    CRVAL      CDELT(microns)  Min & Max (microns)
2022-07-14 08:57:50,928 - stpipe.Spec3Pipeline.cube_build - INFO - Axis 3  2157   1.00  11.55150019   0.00300000  11.55000019  18.02100025
2022-07-14 08:57:50,929 - stpipe.Spec3Pipeline.cube_build - INFO - Cube covers channel, subchannel: 3, short
2022-07-14 08:57:50,930 - stpipe.Spec3Pipeline.cube_build - INFO - Cube covers channel, subchannel: 3, medium
20

2022-07-14 09:04:08,517 - stpipe.Spec3Pipeline.extract_1d - INFO - Saved model in /Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage3/Level3_ch4-mediumshortlong-_x1d.fits
2022-07-14 09:04:08,684 - stpipe.Spec3Pipeline.extract_1d - INFO - Saved model in /Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage3/Level3_ch2-mediumshortlong-_x1d.fits
2022-07-14 09:04:08,851 - stpipe.Spec3Pipeline.extract_1d - INFO - Saved model in /Users/dlaw/FlightData/APT2732_quintet/Data/Obs04/stage3/Level3_ch3-mediumshortlong-_x1d.fits
2022-07-14 09:04:08,852 - stpipe.Spec3Pipeline.extract_1d - INFO - Step extract_1d done
2022-07-14 09:04:08,861 - stpipe.Spec3Pipeline - INFO - Ending calwebb_spec3
2022-07-14 09:04:08,862 - stpipe.Spec3Pipeline - INFO - Step Spec3Pipeline done


In [26]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")

Runtime so far: 1476.0864 seconds


<hr style="border:1px solid gray"> </hr>

<img style="float: center;" src="https://www.stsci.edu/~dlaw/stsci_logo.png" alt="stsci_logo" width="200px"/> 